In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('questions.csv')

In [3]:
df.shape

(404351, 6)

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
# creating new data frame with 3000 rows from df
new_df = df.sample(3000)

In [6]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [7]:
new_df.duplicated()

375337    False
1201      False
351659    False
102133    False
19308     False
          ...  
335760    False
373389    False
251780    False
139799    False
196475    False
Length: 3000, dtype: bool

In [8]:
new_df.duplicated().sum()

0

In [9]:
ques_df = new_df[['question1','question2']]
ques_df.sample(5)

,question1,question2
29479,What goes behind the scene when someone sells ...,How can I find/search a mutual fund with the c...
257073,"How do I use shall and should, will and would,...","How can I determine when to use had, have, had..."
402541,Where is a good place to find women who are av...,Are there feminists out there who believe men ...
317912,"In the Order of precedence, why is a Bharat Ra...","In the Order of precedence, why is a Bharat Ra..."
333087,How can I remove yellow stains from satin tabl...,What can I do to remove chocolate stains from ...


In [ ]:
# using Count Vectorizer

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)
                                              

In [12]:
temp_df1 = pd.DataFrame(q1_arr, index = ques_df.index) # data frame having embeddings of questions from question1
temp_df2 = pd.DataFrame(q2_arr, index = ques_df.index) # data frame having embeddings of questions from question2
temp_df = pd.concat([temp_df1, temp_df2], axis = 1)

In [13]:
temp_df.shape

(3000, 6000)

In [14]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
375337,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
351659,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19308,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335760,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
373389,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
251780,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
139799,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [16]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
375337,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
351659,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19308,0,0,0,0,0,0,0,0,0,0,...,0,2,0,1,0,0,0,0,0,0


In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
accuracy_score(y_test,y_pred)

0.6816666666666666